<a href="https://colab.research.google.com/github/tukamilano/anagram/blob/main/anagram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fugashi unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 8.0 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=cf12a32adb848245e2d97f521d880575a0ef88ce8b31efd2b2acd5e79512f3c1
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [2]:
import requests

# Hugging Face上のファイルURL
url0 = "https://huggingface.co/milano0017/hiragana_KenLM/resolve/main/model.bin"
url1 = "https://huggingface.co/milano0017/hiragana_KenLM/resolve/main/reversed_model.bin"

# 保存先のファイル名
destination_file_name0 = "model.bin"
destination_file_name1 = "reversed_model.bin"

# ファイルをダウンロード
response = requests.get(url0, stream=True)
if response.status_code == 200:
    with open(destination_file_name0, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"'{destination_file_name0}' をダウンロードしました。")
else:
    print(f"エラー: ダウンロードに失敗しました (status code: {response.status_code})")

response = requests.get(url1, stream=True)
if response.status_code == 200:
    with open(destination_file_name1, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"'{destination_file_name1}' をダウンロードしました。")
else:
    print(f"エラー: ダウンロードに失敗しました (status code: {response.status_code})")


'model.bin' をダウンロードしました。
'reversed_model.bin' をダウンロードしました。


In [3]:
!sudo apt-get install -y libboost-all-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libboost-atomic-dev libboost-atomic1.74-dev
  libboost-atomic1.74.0 libboost-chrono-dev libboost-chrono1.74-dev
  libboost-chrono1.74.0 libboost-container-dev libboost-container1.74-dev
  libboost-container1.74.0 libboost-context-dev libboost-context1.74-dev
  libboost-context1.74.0 libboost-coroutine-dev libboost-coroutine1.74-dev
  libboost-coroutine1.74.0 libboost-date-time-dev libboost-date-time1.74-dev
  libboost-date-time1.74.0 libboost-exception-dev libboost-exception1.74-dev
  libboost-fiber-dev libboost-fiber1.74-dev libboost-fiber1.74.0
  libboost-filesystem-dev libboost-filesystem1.74-dev
  libboost-filesystem1.74.0 libboost-graph-dev libboost-graph-parallel-dev
  libboost-graph-parallel1.74-dev libboost-graph-parallel1.74.0
  libboost-graph1.74-dev libboost-graph1.74.0 libboost-iostreams-dev
  libboost-iost

In [4]:
import os

# Clone KenLM if not already cloned
if not os.path.exists("kenlm"):
    !git clone https://github.com/kpu/kenlm.git
    print("KenLM repository cloned.")
else:
    print("KenLM repository already exists.")

# Build KenLM
# Change to kenlm directory
%cd kenlm

# Create build directory and navigate into it
!mkdir -p build
%cd build

# Configure and build KenLM
!cmake ..
!make -j $(nproc)

# Navigate back to the original content directory
%cd /content

print("KenLM build process complete. You can now try to run the lmplz command.")

Cloning into 'kenlm'...
remote: Enumerating objects: 14185, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 14185 (delta 117), reused 80 (delta 80), pack-reused 14010 (from 2)
Receiving objects: 100% (14185/14185), 5.91 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (8059/8059), done.
KenLM repository cloned.
/content/kenlm
/content/kenlm/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
CMake Warning (dev) at CMakeLis

In [5]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     \ 553.6 kB 4.6 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp312-cp312-linux_x86_64.whl size=3187999 sha256=9ea8242f02a167b96ee9ecf2cfd2f67f90bad581e75a33b9db65bdfed70b6eda
  Stored in directory: /tmp/pip-ephem-wheel-cache-rppi06pv/wheels/92/c8/12/56d187154e078f0eaa74d059017fc1afe1c4d91fbce02ce8d9
Successfully built kenlm


In [6]:
import kenlm

model = kenlm.Model("model.bin")

In [7]:
import fugashi

# ==== 初期化 ====
print("形態素解析器を初期化中...")
tagger = fugashi.Tagger()

形態素解析器を初期化中...


In [40]:
import random
import math
from itertools import combinations

# ===============
# ひらがなセット
# ===============
jp = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")

# ===============================
# Perplexityを計算する関数
# ===============================
def calculate_perplexity(sentence, morph_num_weight=0):
    """
    KenLM を使って
    与えられた文の Perplexity（自然さ）を算出する。
    sentence: もしくは str
    """
    sentence = ''.join(sentence)
    processed_sentence_list = [word.surface for word in tagger(sentence)]
    reversed_processed_sentence_list = processed_sentence_list[::-1]
    processed_sentence = ' '.join(processed_sentence_list)
    reversed_processed_sentence = ' '.join(reversed_processed_sentence_list)
    ppl = -model.score(processed_sentence) -model.score(reversed_processed_sentence)
    #形態素の数にペナルティを与える
    ppl += morph_num_weight * len(processed_sentence_list)
    return ppl

In [47]:
def improve_by_4_opt(sentence, step=1000000):
    v = calculate_perplexity(sentence)
    calculate_num = 0
    improved = True

    while improved:
        improved = False
        indices = list(range(len(sentence)))
        base = list(combinations(indices, 4))
        random.shuffle(base)  # 組の順序をランダム化

        for four_var in base:
            a, b, c, d = four_var
            new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
            new_v = calculate_perplexity(new_sentence)
            calculate_num += 1

            if new_v < v:
                v = new_v
                sentence = new_sentence
                improved = True
                print(f"Improved! step={calculate_num}, new_ppl={v:.3f}")
                break

            if calculate_num >= step:
                print(f"Reached step limit ({step}). Stop.")
                return sentence

    print("No more improvement.")
    return sentence

def improve_by_4_opt_sa(sentence, step=1_000_000, T0=1000.0, Tmin=0.1, alpha=0.99999):
    v = calculate_perplexity(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        indices = sorted(random.sample(range(len(sentence)), 4))
        a, b, c, d = indices
        new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_perplexity(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_ppl={v:.3f}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            print(f"Step {step_num}: T={T:.3f}, ppl={v:.3f}")

    print(f"Finished all {step} steps.")
    print(f"Best perplexity: {best_v:.3f}")
    return best_sentence

def improve_by_4_opt_sa_select_by_morph(sentence, step=1_000_000, T0=1000.0, Tmin=0.1, alpha=0.99999, morph_ratio=0.99):
    v = calculate_perplexity(sentence)
    best_sentence, best_v = sentence, v
    T = T0

    for step_num in range(step):
        # ランダムに4点選択
        if random.random() < morph_ratio:
          processed_sentence_list = [word.surface for word in tagger(''.join(sentence))]
          indices = sorted(random.sample(range(len(processed_sentence_list)), 4))
          a, b, c, d = indices
          new_processed_sentence_list = processed_sentence_list[:a] + processed_sentence_list[c:d] + processed_sentence_list[b:c] + processed_sentence_list[a:b] + processed_sentence_list[d:]
          new_sentence = ''.join(new_processed_sentence_list)
        else:
          indices = sorted(random.sample(range(len(sentence)), 4))
          a, b, c, d = indices
          new_sentence = sentence[:a] + sentence[c:d] + sentence[b:c] + sentence[a:b] + sentence[d:]
        new_v = calculate_perplexity(new_sentence)
        delta = new_v - v

        # 改善 or 確率的悪化許容
        if delta < 0 or random.random() < math.exp(-delta / T):
            sentence, v = new_sentence, new_v
            if v < best_v:
                best_sentence, best_v = sentence, v
                print(f"Step {step_num}: Improved! new_ppl={v:.3f}")

        # 温度を減衰
        T = max(T * alpha, Tmin)

        # ログ出力（確認用）
        if step_num % 1000 == 0:
            print(f"Step {step_num}: T={T:.3f}, ppl={v:.3f}")

    print(f"Finished all {step} steps.")
    print(f"Best perplexity: {best_v:.3f}")
    return best_sentence

# ========================
# 実行部分
# ========================
def main():
    sentence = jp[:]  # コピーしておく
    random.shuffle(sentence)
    print("Initial sentence:", "".join(sentence))
    improved_sentence = improve_by_4_opt_sa_select_by_morph(sentence)
    print("Final sentence:", "".join(improved_sentence))

if __name__ == "__main__":
    main()


Initial sentence: へなおねひはめゆやさしいこふせつけわちよますのをむんとたにらてかぬうるえそろみほきくりれあも
Step 0: Improved! new_ppl=245.051
Step 0: T=999.990, ppl=245.051
Step 1: Improved! new_ppl=235.781
Step 2: Improved! new_ppl=226.264
Step 6: Improved! new_ppl=225.589
Step 7: Improved! new_ppl=224.442
Step 8: Improved! new_ppl=220.368
Step 15: Improved! new_ppl=218.999
Step 19: Improved! new_ppl=217.703
Step 24: Improved! new_ppl=217.107
Step 35: Improved! new_ppl=210.004
Step 36: Improved! new_ppl=209.434
Step 38: Improved! new_ppl=209.294
Step 40: Improved! new_ppl=207.798
Step 41: Improved! new_ppl=204.398
Step 42: Improved! new_ppl=203.292
Step 43: Improved! new_ppl=201.768
Step 66: Improved! new_ppl=198.441
Step 81: Improved! new_ppl=198.170
Step 82: Improved! new_ppl=197.413
Step 83: Improved! new_ppl=194.253
Step 85: Improved! new_ppl=193.808
Step 86: Improved! new_ppl=189.532
Step 88: Improved! new_ppl=189.490
Step 89: Improved! new_ppl=189.238
Step 90: Improved! new_ppl=187.643
Step 351: Improved! new_ppl=184.134

In [48]:
text = "はるさめやゆきてんようへこますせつねえあわれみおいぬけろなしほかそのふたりをにらむひとくちも"
processed_sentence = ' '.join(word.surface for word in tagger(text))
print(processed_sentence)

はるさめ や ゆき てん よう へこます せつねえ あわれみ おいぬけろ なし ほか その ふたり を にらむ ひとくち も
